# feed_forward 

In [99]:
import math

# The sigmoid activation function.
def activation_function(x):
    return 1 / (1 + math.exp(-x))

In [100]:
class Neuron:
    def __init__(self, name):
        self.name = name
        self.inputs = []
        self.outputs = []
        self.sum = 1   # Set sum and value to 1 (mostly for bias neurons).
        self.output = 1

    # Use our input synapses to set our value.
    def set_value(self):
        # Don't change the bias neuron values
        if self.name == "Bias":
            return
        
        self.sum = 0
        
        # Add up the inputs
        print("inputs length " + str(len(self.inputs)))
        for input in self.inputs:
            
            self.sum += input.weight
        print(self.sum)
        self.output = activation_function(self.sum)
        print('output ' + str(self.output))
        

In [101]:
class Synapse:
    def __init__(self, from_neuron, to_neuron, weight):
        self.from_neuron = from_neuron
        self.to_neuron = to_neuron
        self.weight = weight
        
        if from_neuron is not None:  # This happens if this is a bias neuron.
            
            from_neuron.outputs.append(self)
        # This list should be as big as the width of any one of the layers.
        # This is the to_neuron passed in not the self one. so the to_neuron is a global variable added to in each neuron constructor
        
        to_neuron.inputs.append(self)

In [102]:
class NeuralNet:
    # layer_sizes is a list holding the number of
    # neurons (not counting biases) in the layers.
    def __init__(self, layer_sizes):
        # Build the layers.
        self.all_layers = [] # holds all layers each as a list
        num_layers = len(layer_sizes)

        for i in range(num_layers):
            add_bias = i < num_layers - 1  # No bias in the output layer.
            # print(add_bias)
            self.all_layers.append(self.build_layer(layer_sizes[i], f'Neuron_{i}', add_bias))
            


        # Make shortcuts to the input and output layers.
        self.input_layer = self.all_layers[0]
        self.output_layer = self.all_layers[-1]

        # Make synapses between layers.
        for layer_num in range(num_layers - 1):
            self.build_synapses(
                self.all_layers[layer_num],
                self.all_layers[layer_num + 1])

    # Make a list of {num_neurons} neurons.
    # Give them names {base_name}_{i} where {i} is the neuron number.
    # If {add_bias} is True, add a bias neuron.
    def build_layer(self, num_neurons, base_name, add_bias):
        layer = []
        for i in range(num_neurons):
           
            layer.append(Neuron(base_name + "_" + str(i)))
            
        if add_bias:
            layer.append(Neuron("Bias"))
        
        return layer
        
                

    # Make links connecting two neuron layers.
    def build_synapses(self, from_layer, to_layer):
        layer = []
        # for every Neuron draw synapses to every Neuron in the layer below
        for from_layer_neuron in from_layer:
            
            for to_layer_neuron in to_layer:
                if not to_layer_neuron.name == "Bias":
                    layer.append(Synapse(from_layer_neuron, to_layer_neuron, from_layer_neuron.output))
                    
            # for item in layer:
            #     print(item.weight)        
                    
                # for every node in to layer
                
                

    # Run the network on these inputs.
    # Return a list of output values.
    def evaluate(self, input_values):
        

        if(not len(input_values) + 1 == len(self.input_layer)):
            raise TypeError('Number of input values does not match the size of the input layer')
        result_lst = []
        
        
        print("input values length " + str(len(input_values)))
        for i in range(len(input_values)):
            self.input_layer[i].output = input_values[i]
            
        # print('input layer length')
        # print(str(len(self.input_layer)))
        for i in range(len(self.input_layer)):
            # print('input layer value')
            print(self.input_layer[i])
            
            
        for i in range(1, len(self.all_layers)):
            for neuron in self.all_layers[i]:
                neuron.set_value()
            
            
        for i in range(len(self.output_layer)):
            result_lst.append(self.output_layer[i].output)
        
        return result_lst
        

    # Print the neuron names and synapse weights.
    def dump(self):
        for layer_num in range(len(self.all_layers)):
            print(f'Layer {layer_num}:')
            for neuron in self.all_layers[layer_num]:
                print(f'    {neuron.name}: ', end='')
                for synapse in neuron.outputs:
                    print(f'{synapse.weight:>7.2f} ', end='')
                print()

In [103]:
# Functions to draw the network.
NEURON_RADIUS = 20

# Position the neurons for drawing.
def position_neurons(network, canvas):
    # Get the canvas's dimensions.
    canvas.winfo_toplevel().update()
    width = canvas.winfo_width()
    height = canvas.winfo_height()

    # Find the maximum number of neurons in any layer.
    max_neurons = len(max(network.all_layers, key=len))

    # Calculate some spacing values.
    margin = NEURON_RADIUS + 5
    neuron_gap = (height - 2 * margin) / (max_neurons - 1)  # Spacing between neurons in a layer.
    layer_gap = (width - 2 * margin) / (len(network.all_layers) - 1)  # Spacing between layers.

    # Position the neurons.
    x = margin
    for layer in network.all_layers:
        neuron_gap = (height - 2 * margin) / (len(layer) - 1)  # Spacing between neurons in a layer.
        y = (height - neuron_gap * (len(layer) - 1)) / 2
        for neuron in layer:
            neuron.x = x
            neuron.y = y
            y += neuron_gap
        x += layer_gap

# Draw the network's synapses.
def draw_synapses(network, canvas):
    # Inputs.

    for layer in network.all_layers:
        # Draw the synapse lines.
        for neuron_num in range(len(layer)):
            from_neuron = layer[neuron_num]
            for synapse in from_neuron.outputs:
                draw_synapse_line(synapse, canvas, neuron_num)

        # Draw the synapse weights on top of the lines.
        for neuron_num in range(len(layer)):
            
            from_neuron = layer[neuron_num]
            
            for synapse in from_neuron.outputs:
                draw_synapse_weight(synapse, canvas, neuron_num)

# Draw a synapse's line.
def draw_synapse_line(synapse, canvas, neuron_num):
    from_neuron = synapse.from_neuron
    to_neuron = synapse.to_neuron

    # Draw the line.
    canvas.create_line(
        from_neuron.x, from_neuron.y,
        to_neuron.x, to_neuron.y,
        fill='blue')

# Draw a synapse's weight.
def draw_synapse_weight(synapse, canvas, neuron_num):
    from_neuron = synapse.from_neuron
    to_neuron = synapse.to_neuron

    # Draw the weight text.

    x = (5 * from_neuron.x + 1 * to_neuron.x) / 6
    y = (5 * from_neuron.y + 1 * to_neuron.y) / 6

    print('x ' + str(x))
    print('y ' + str(y))
    text_tag = canvas.create_text(x, y, text=f'{synapse.weight:.2f}')
    print('txt tag')
    print(text_tag)
    # Clear an area for the text.
    bounds = canvas.bbox(text_tag)  # Returns (x1, y1, x2, y2)
    print('bounds')
    for b in bounds:
        print(b)
    canvas.create_rectangle(*bounds, fill='white', outline='white')

    # Lift the label above the cleared area.
    canvas.tag_raise(text_tag)

# Draw the network's neurons.
def draw_neurons(network, canvas):
    for layer in network.all_layers:
        for neuron in layer:
            draw_neuron(neuron, canvas)

# Draw a neuron.
def draw_neuron(neuron, canvas):
    width = 1
    if neuron.name.startswith('Bias'):
        width = 3

    color = 'silver'  # For uncomitted neurons.
    if neuron.name.startswith('Bias'):
        color = 'pink'
    elif neuron.output < 0.33:
        color = 'white'
    elif neuron.output > 0.67:
        color = 'lightgreen'

    canvas.create_oval(
        neuron.x - NEURON_RADIUS, neuron.y - NEURON_RADIUS,
        neuron.x + NEURON_RADIUS, neuron.y + NEURON_RADIUS,
        fill=color, outline='black', width=width)
    # Draw the neuron's value.
    canvas.create_text(neuron.x, neuron.y, text=f'{neuron.output:.2f}')
    # Draw the neuron's sum and value.
    #canvas.create_text(neuron.x, neuron.y, text=f'{neuron.sum:.2f}\n{neuron.output:.2f}')

# Make a simple drawing of the network.
def draw_network(network, canvas):
    canvas.delete('all')

    # Position the neurons.
    position_neurons(network, canvas)

    # Draw the synapses.
    draw_synapses(network, canvas)

    # Draw the neurons.
    draw_neurons(network, canvas)

In [104]:
import tkinter as tk

# Geometry constants.
WINDOW_WID = 910
WINDOW_HGT = 610
FRAME1_WID = 100
PADX = 5
NETWORK_CANVAS_WID = WINDOW_WID - FRAME1_WID - 4 * PADX
NETWORK_CANVAS_HGT = WINDOW_HGT - 2 * PADX

class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.network = None

        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('feed_forward')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry(f'{WINDOW_WID}x{WINDOW_HGT}')

        # Load the data.
        self.load_data()

        # Build the UI.
        self.build_ui()

        # Draw the network.
        self.redraw_network()

        # Display the window.
        self.window.focus_force()
        self.window.mainloop()

    # Build an odd/even test network.
    def load_data(self):
        self.network = NeuralNet([3,4,2])
        weights = [
        [
            # Input layer.
            [ -100, 10, 10, -100],  # Neuron 0
            [ -100, 100, 10, 10],  # Neuron 1
            [ -100, 10, -100, 10],  # Neuron 2
            [ 5, -15, -15, -15],  # Bias
        ],
        [
            # Hidden layer.
            [ -10, 10],  # Neuron 0
            [ -10, 10],  # Neuron 1
            [ 10, -10],  # Neuron 2
            [ -10, 10],  # Neuron 3
            [ 5, 5],  # Bias
        ]
        ]
        self.set_synapse_weights(self.network.all_layers, weights)
        

    # Set the weights for the synapses in all layers.
    def set_synapse_weights(self, all_layers, weights):
        for layer_num in range(len(all_layers)):
            layer = all_layers[layer_num]
            
            for neuron_num in range(len(layer)):
                neuron = layer[neuron_num]
                
                for synapse_num in range(len(neuron.outputs)):
                    synapse = neuron.outputs[synapse_num]
                    synapse.weight = weights[layer_num][neuron_num][synapse_num]
                    print('synapse weight ' + str(synapse.weight))
                    


    def build_ui(self):
        # Make controls to define the network.
        frame1 = tk.Frame(self.window, width=FRAME1_WID)
        frame1.pack(side=tk.LEFT, expand=False, fill=tk.Y, padx=PADX)
        frame1.pack_propagate(False)

        frame2 = tk.Frame(self.window)
        frame2.pack(side=tk.LEFT, expand=True, fill=tk.BOTH, padx=PADX)

        # Checkbuttons.
        check_frame = tk.Frame(frame1, width=200)
        check_frame.pack(side=tk.TOP)

        self.check0_value = tk.IntVar()
        check0 = tk.Checkbutton(check_frame, variable=self.check0_value,
            onvalue=1, offvalue=0, command=self.redraw_network)
        check0.pack(side=tk.LEFT)

        self.check1_value = tk.IntVar()
        check1 = tk.Checkbutton(check_frame, variable=self.check1_value,
            onvalue=1, offvalue=0, command=self.redraw_network)
        check1.pack(side=tk.LEFT)

        self.check2_value = tk.IntVar()
        check2 = tk.Checkbutton(check_frame, variable=self.check2_value,
            onvalue=1, offvalue=0, command=self.redraw_network)
        check2.pack(side=tk.LEFT)

        # Labels to display results.
        self.even_value = tk.StringVar()
        self.even_value.set('Even = True')
        even_label = tk.Label(frame1, textvariable=self.even_value, height=1)
        even_label.pack(side=tk.TOP)

        self.odd_value = tk.StringVar()
        self.odd_value.set('Odd = False')
        odd_label = tk.Label(frame1, textvariable=self.odd_value, height=1)
        odd_label.pack(side=tk.TOP)

        # Frame 2.
        # Network canvas.
        self.canvas = tk.Canvas(frame2, bg='white',
            borderwidth=0, highlightthickness=0, relief=tk.SUNKEN,
            width=NETWORK_CANVAS_WID, height=NETWORK_CANVAS_HGT)
        self.canvas.pack(side=tk.LEFT, anchor=tk.NW)

    # Redraw the network after the user changed a checkbutton.
    def redraw_network(self):
        # Evaluate with the current checkbutton values.
        input_values = [
            self.check0_value.get(),
            self.check1_value.get(),
            self.check2_value.get(),
        ]
        result = self.network.evaluate(input_values)
        # print(len(result))
        # for r in result:
        #     print(r)

        # Display the result (for debugging).
        self.network.dump()
        print()

        # Display the results textually.
        if result[0] < 0.33:
            self.odd_value.set('Odd = False')
        elif result[0] > 0.67:
            self.odd_value.set('Odd = True')
        else:
            self.odd_value.set('??????')

        if result[1] < 0.33:
            self.even_value.set('Even = False')
        elif result[1] > 0.67:
            self.even_value.set('Even = True')
        else:
            self.even_value.set('??????')

        # Draw the network.
        draw_network(self.network, self.canvas)

    def kill_callback(self):
        self.window.destroy()

In [105]:
App()

synapse weight -100
synapse weight 10
synapse weight 10
synapse weight -100
synapse weight -100
synapse weight 100
synapse weight 10
synapse weight 10
synapse weight -100
synapse weight 10
synapse weight -100
synapse weight 10
synapse weight 5
synapse weight -15
synapse weight -15
synapse weight -15
synapse weight -10
synapse weight 10
synapse weight -10
synapse weight 10
synapse weight 10
synapse weight -10
synapse weight -10
synapse weight 10
synapse weight 5
synapse weight 5
input values length 3
inputs length 4
-295
output 7.640606587007545e-129
inputs length 4
105
output 1.0
inputs length 4
-95
output 5.5210822770285325e-42
inputs length 4
-95
output 5.5210822770285325e-42
inputs length 5
-15
output 3.059022269256247e-07
inputs length 5
25
output 0.999999999986112
Layer 0:
    Neuron_0_0: -100.00   10.00   10.00 -100.00 
    Neuron_0_1: -100.00  100.00   10.00   10.00 
    Neuron_0_2: -100.00   10.00 -100.00   10.00 
    Bias:    5.00  -15.00  -15.00  -15.00 
Layer 1:
    Neuron_1